In [4]:
import os
import pandas as pd
from models import prep

pd.set_eng_float_format(accuracy=2, use_eng_prefix=True)

df = prep.load_dataset()
df = prep.split_sum_totals()
df = prep.feature_extraction(df)
df = prep.remove_obvious_outliers(df)

In [3]:
# forget about the time related features 
df = df.drop(columns=["id_lsf", "sinDayOfYear", "cosDayOfYear",
                      "daysSinceBaseDate"])

In [4]:
for agent in ["id_pa", "id_be"]:
    global_stats = pd.concat([
        df.groupby(agent)["importo"].median().rename(
            agent.strip("id_") + "_global_median"),
        df.groupby(agent)["importo"].std().rename(
            agent.strip("id_") + "_global_std")], axis=1)
    df = df.join(global_stats, on=agent)

# sta cosa è sostituta da median_annual_contract e med_ann_rev


In [5]:
df = df[df["cpv"].isin(prep.abc_cpv_short_names.keys())]
df = df[df["id_scelta_contraente"].isin(prep.abc_procedure_short_names.keys())]

In [6]:
# df must be local
for agent in ["id_pa", "id_be"]:
    local_stats = pd.concat([
        df.groupby(agent)["importo"].median().rename(
            agent.strip("id_") + "_local_median"),
        df.groupby(agent)["importo"].std().rename(
            agent.strip("id_") + "_local_std")], axis=1)
    df = df.join(local_stats, on=agent)

In [7]:
df[[
    "importo", "pa_local_median", "be_local_median", "pa_med_ann_contr", 
    "be_med_ann_contr", "pa_med_ann_expenditure", "be_med_ann_revenue", 
    "id_scelta_contraente", "cpv", "id_pa", "id_be"]]

,importo,pa_local_median,be_local_median,pa_med_ann_contr,be_med_ann_contr,pa_med_ann_expenditure,be_med_ann_revenue,id_scelta_contraente,cpv,id_pa,id_be
id_lotto,,,,,,,,,,,
17453498,707.92,809.26,765.00,1.04k,917.28,101.71M,246.28k,23,33,2997,95012
5513992,29.15k,29.07k,170.00k,3.29k,149.73k,20.04M,4.18M,23,45,592,103976
2251319,128.16,5.00k,4.20k,4.73k,1.00k,208.54M,21.79k,23,33,2992,283851
2250258,5.00k,6.60k,1.88k,1.02k,2.07k,6.93M,821.93k,23,45,14374,94395
3316126,340.80k,3.62k,39.69k,4.41k,28.99k,228.08M,15.15M,1,33,2435,93140
...,...,...,...,...,...,...,...,...,...,...,...
3051186,4.70k,1.13k,815.00,1.06k,576.75,102.56M,263.19k,23,33,2998,85233
6292719,814.50,600.00,800.00,728.00,863.52,18.90M,393.50k,23,33,25911,89351
3050262,39.00k,20.00k,1.81k,19.70k,2.53k,632.96M,474.75k,26,33,2987,94771


idea: uni-dimentional clustering within the cpv-procedures clusters

In [ ]:
df["id_pa"] == 2997

In [10]:
# where lies the lot in the (pa, be) distribution?
import seaborn as sns
import numpy as np
sns.histplot(x=np.log10(df[df["id_pa"] == 2997].importo), y=np.log10(df[df["id_be"] == 95012].importo))

/Users/nepal/miniconda3/lib/python3.9/site-packages/seaborn/_core.py:927: FutureWarning: reindexing with a non-unique Index is deprecated and will raise in a future version.
  plot_data = pd.DataFrame(plot_data)


ValueError: cannot reindex on an axis with duplicate labels

quindi ci sono doppioni ed è corretto poiché nel proprocessing non li ho considerati 

In [31]:
# FROM prep.py
import pandas as pd
import numpy as np
from os import path

import_directory = "synData6July"
lotti_fn = "export_lotti_veneto_2016_2018_giulio_v2.csv"
vincitori_fn = "export_vincitori_veneto_2016_2018_giulio_v2.csv"
output_directory = "datasets"


def replace_missing_value(df, col, replacement_col):
    mask = df[replacement_col].notna() & df[col].isna()
    df.loc[df[mask].index, col] = df[replacement_col][mask]
    return df

lotti = pd.read_csv(
    path.join(import_directory, lotti_fn))
vincitori = pd.read_csv(
    path.join(import_directory, vincitori_fn))

# convert datatypes
lotti.data_inizio = pd.to_datetime(lotti.data_inizio, yearfirst=True)
lotti.data_fine = pd.to_datetime(lotti.data_fine, yearfirst=True)
lotti.data_inferita = pd.to_datetime(lotti.data_inferita, yearfirst=True)

# replace missing values in col1 with values from col2
lotti = replace_missing_value(lotti, "importo", "importo_base_asta")
lotti = replace_missing_value(lotti, "data_inizio", "data_inferita")

# drop table attributes with mostly missing values
missingValuesCols = [
    "oggetto", "importo_liquidato", "importo_base_asta", "data_inferita",
    "id_mod_realizz", "cpv_vero"]
lotti = lotti.drop(columns=missingValuesCols)
# print("columns dropped with mostly missing values:")
# print(str(missingValuesCols))

# clean the dfs from the remaining missing values
lotti = lotti.dropna()
vincitori = vincitori.dropna()

# print("dropped all the rows with at least one missing value")
# cast to int64 cols now w/out np.nan
lotti.id_scelta_contraente = lotti.id_scelta_contraente.astype('int')
lotti.id_lsf = lotti.id_lsf.astype('int')
lotti.cpv = lotti.cpv.astype('int')
# drop columns that lead to sparse matrices
# following tests: try models that handle sparse datasets
lotti = lotti.drop(
    columns=["id_forma_giuridica", "uber_forma_giuridica"])
vincitori = vincitori.drop(
    columns=["id_forma_giuridica", "uber_forma_giuridica"])

# AGGIUNGERE DIVISIONE importo tra lotti vinti da più di un be


# merge the datasets

In [122]:
df = lotti.merge(vincitori, on="id_lotto")

In [123]:
df[df.duplicated(keep=False)].sort_values(by="importo").sort_values(by="id_be")

,id_lotto,id_pa,id_scelta_contraente,importo,data_inizio,data_fine,id_lsf,cpv,id_be
727594,508923150,21033,23,6.40k,2017-03-29,2018-06-29,2,72,84527
727592,508923150,21033,23,6.40k,2017-03-29,2018-06-29,2,72,84527
727774,4768706,25251,26,5.79k,2017-02-16,2017-02-16,3,30,84755
727775,4768706,25251,26,5.79k,2017-02-16,2017-02-16,3,30,84755
727846,3331711,21033,23,147.00,2016-10-29,2016-11-30,3,37,86987
...,...,...,...,...,...,...,...,...,...
727795,1444519,13081,4,580.00k,2017-05-29,2018-03-01,1,45,527659
728198,2802476,21033,23,182.00,2016-09-20,2016-09-28,2,37,835507
728196,2802476,21033,23,182.00,2016-09-20,2016-09-28,2,37,835507
727717,17014950,18869,8,32.07k,2018-09-12,2022-12-31,2,71,1129915


In [124]:
# remove duplicates
df = df[~df.duplicated()]

(755757, 9)
(755660, 9)


In [125]:
# considera solo i lotti aventi più di un vincitore
temp = df[df["id_lotto"].duplicated(keep=False)]

In [126]:
temp = temp.join(temp.groupby("id_lotto").size().rename("n_winners"), on="id_lotto")

In [127]:
temp = temp["importo"] / temp["n_winners"]

In [128]:
# 3 is the positional index of the "importo" column
df.iloc[temp.index, 3] = temp.values

In [129]:
df.duplicated().sum()

0

In [130]:
df

,id_lotto,id_pa,id_scelta_contraente,importo,data_inizio,data_fine,id_lsf,cpv,id_be
0,15254532,27067,23,3.87k,2017-03-09,2017-03-09,2,63,118170
1,15001253,27385,23,2.31k,2017-08-21,2017-08-21,3,22,332080
2,8649252,7068,23,156.00,2016-12-19,2016-12-31,3,22,93303
3,7971134,20084,23,176.00,2016-08-05,2016-08-05,2,80,343041
4,17453498,2997,23,707.92,2018-05-31,2018-05-31,3,33,95012
...,...,...,...,...,...,...,...,...,...
755752,4365772,35871,23,19.80k,2016-06-06,2016-08-01,1,45,180193
755753,3096075,20656,23,1.28k,2017-05-26,2017-05-26,2,63,409747
755754,6459810,582,23,2.19k,2018-02-15,2018-03-30,3,42,96744
755755,3096334,6556,23,26.56,2016-12-02,2016-12-31,3,32,84800
